##Auto backing up absa ansotations

1.   absa anotation: label studio  from 9091-9100 ports 
2.   Saved at https://drive.google.com/drive/folders/1REUAhPUY8NzOXaj3OtPosOIZsxC8k00l



In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import pickle
import json
import random
import numpy as np
import pandas as pd
import getpass, os
import sys
''' if using casa'''
# username = getpass.getpass("Github username: ")
# password = getpass.getpass("Github password: ")
# os.environ["CRED"] = f"{username}:{password}"
# !git clone https://${CRED}@github.com/lopentu/casa
# del username, password
# !pip install -r /content/casa/requirements.txt

' if using casa'

In [50]:
# from import_casa import casa
# sys.path = ["/content/casa/src"] + sys.path
# import casa

##Config 
Waiting for other annotators to be listed


In [51]:
# request url = 'http://140.112.147.132:{port}/api/project/export?format=JSON'
config = {
    'port dict': {
    9091:'Andrea',
    9092:'品而',
    9093:'Amy',
    9094:'Nana',
    9095: '子茹',
    9096: '聯輝',
    9097: '子娟'}, # if no tasks are done, no files will be saved for the port
    'port range':range(9091, 9100+1),   
    # range(9091, 9100+1)
}

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import requests, zipfile, io
from datetime import date

PATH = '/content/drive/MyDrive/指向情緒案/data/annot_data'
BKUP_DIR = os.path.join(PATH, 'annotated_data_bkup')
os.makedirs(BKUP_DIR, exist_ok = True)
PATH = os.path.join(PATH, BKUP_DIR)

def CreateTodayDir(PATH):
  today = date.today()
  TARGET_DIR = today.strftime("%Y%m%d")
  os.makedirs(TARGET_DIR, exist_ok = True)
  new_dir = os.path.join(BKUP_DIR, TARGET_DIR)
  return new_dir

def SaveJSON(config, PATH):
  annots = []
  new_dir = CreateTodayDir(PATH)
  print('Today\'s dir:', new_dir)
  for port in config['port range']:
    url = f'http://140.112.147.132:{port}/api/project/export?format=JSON'
    r = requests.get(url, allow_redirects=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.filelist[0].filename = f'result_{port}.json'
    z.extract(z.filelist[0], new_dir)
    JSON_PATH = os.path.join(new_dir, f'result_{port}.json')
    with open(JSON_PATH, "r", encoding="UTF-8") as fin:
      jsonfile = json.load(fin)
      if len(jsonfile) != 0: 
        annots.append(jsonfile)
  return annots

In [54]:
annots = SaveJSON(config, PATH)
print("Saved {} json files.".format(len(annots)))
# print("Progress:")
Table = list()
for i, annot in enumerate(annots, start = 1):
  idx = 9090 + i
  annotator = config['port dict'][idx]
  # print(f'{annotator}: {len(annot)}')
  annotatorDict = dict()
  annotatorDict['Annotator'] = annotator
  annotatorDict['Tasks submitted'] = len(annot)
  Table.append(annotatorDict)

Today's dir: /content/drive/MyDrive/指向情緒案/data/annot_data/annotated_data_bkup/20210320
Saved 4 json files.


In [55]:
def extract_result(annot_item, port):
    serial = annot_item["data"]["serial"]
    thread_idx = annot_item["data"]["thread_idx"]
    result = annot_item["completions"][0]["result"]
    entries = []
    for x in result:
        from_name = x.get("from_name", "")
        ent_type = x["type"]
        value_dict = x.get("value", {})
        ent_choice = value_dict.get("choices", [""])[0]
        ent_text = value_dict.get("text", "")
        if isinstance(ent_text, list):
            ent_text = "".join(ent_text)
        entries.append((
            port,
            thread_idx, serial,
            from_name, ent_type, ent_choice, ent_text
        ))   
    return entries

In [56]:
colnames = "port,thread_idx,serial,from_name,ent_type,ent_choice,ent_text".split(",")
def convert_annot_result(annot_obj, port):
    entries = []
    for annot_x in annot_obj:
        entries.extend(extract_result(annot_x, port))
    return pd.DataFrame(entries, columns=colnames)

In [57]:
list(annot[0].keys())

['completions', 'data', 'id']

In [58]:
def SaveCSV(config = config, annots = annots, PATH = PATH):
  port_list = list(config['port range'])
  # start = port_list[0]
  dframes = []
  for idx, annot in enumerate(annots): 
    '''
    annots: everyone's annotations
    annot: one annotator's annotations
    annot[0]: one task in annot
    '''
    port = port_list[idx]
    df = convert_annot_result(annot, port)
    PATH = CreateTodayDir(PATH)
    dframes.append(df)
    df.to_csv(f"{PATH}/annot_frame_{port}.csv")
  return dframes 

In [59]:
print("Saving csv...")
dframes = SaveCSV(config, annots, PATH)
print("Saved {} csv files.".format(len(annots)))
for i, df in enumerate(dframes, start = 1): 
  idx = 9090 + i
  annotator = config['port dict'][idx]
  # print(f'{annotator}:{len(df)}')
  AnnotatorDict = next((Dict for Dict in Table if Dict['Annotator'] == annotator), None)
  AnnotatorDict['Tags Annotated'] = len(df)

Saving csv...
Saved 4 csv files.


In [60]:
# ShowTable
df = pd.DataFrame(Table)
df

,Annotator,Tasks submitted,Tags Annotated
0,Andrea,50,363
1,品而,63,843
2,Amy,32,157
3,Nana,52,365


In [61]:
annot[0]["completions"]

[{'created_at': 1615984403,
  'id': 51001,
  'lead_time': 59.42,
  'result': [{'from_name': 'aspect-label',
    'id': 'nslgQu9cZ9',
    'to_name': 'text',
    'type': 'hypertextlabels',
    'value': {'end': '/div[1]/ol[1]/li[1]/text()[1]',
     'endOffset': 8,
     'htmllabels': ['Entity'],
     'start': '/div[1]/ol[1]/li[1]/text()[1]',
     'startOffset': 6,
     'text': '中華'}},
   {'from_name': 'ent-norm',
    'id': 'nslgQu9cZ9',
    'to_name': 'text',
    'type': 'choices',
    'value': {'choices': ['中華電信'],
     'end': '/div[1]/ol[1]/li[1]/text()[1]',
     'endOffset': 8,
     'start': '/div[1]/ol[1]/li[1]/text()[1]',
     'startOffset': 6,
     'text': '中華'}},
   {'from_name': 'aspect-label',
    'id': 'OvqNYiPZYv',
    'to_name': 'text',
    'type': 'hypertextlabels',
    'value': {'end': '/div[1]/ol[1]/li[1]/text()[2]',
     'endOffset': 5,
     'htmllabels': ['Attribute'],
     'start': '/div[1]/ol[1]/li[1]/text()[2]',
     'startOffset': 2,
     'text': '469'}},
   {'from_name

In [62]:
# import html
# print(html.unescape(text))